
# AutoGen + LangChain Use Case - Uniswap Protocol AI Agent

AutoGen is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation. They are adaptable to different operation modes that encompass the utilization of LLMs, human inputs, and various tools.

LangChain is an open-source framework designed for software developers engaged in AI and ML. It enables them to seamlessly integrate LLM with external components, facilitating the creation of LLM-driven applications. The primary aim of LangChain is to establish connections between LLMs such as OpenAI's GPT-3.5 and GPT-4 and various external data sources, enabling the development and utilization of NLP applications.

Both of them are playing key roles in the LLM application development.

AutoGen doesn't support connecting to various external data sources natively. This is exactly where LangChain can come into play.
Use Case - Uniswap Protocol

Uniswap is a decentralized exchange that allows users to trade Ethereum-based tokens.

In this tutorial, I will walk you through the steps to follow to build an AI agent that can complete tasks that involve Uniswap knowledge.

    Build up a vector store with Uniswap V3 whitepaper.
    Set up a conversational retrieval QA chain by LangChain, based on the vector store.
    Define a function answer_uniswap_question by taking a parameter question, calling the QA chain to answer it.
    Set up AutoGen user agent and assistant agent with function calling enabled.

    In the function mapping, the function defined in step 3 is included.

    The assistant agent is instructed by the following message to call the function answer_uniswap_question to answer Uniswap related questions.

    I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

    1. What is Uniswap?
    2. What are the main changes in Uniswap version 3?
    3. How to use Uniswap?

    Start the work now.

    The function call is done by user agent.



In [1]:
import autogen

# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-4"],
#     },
# )
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

# gpt4_api_key = config_list[0]["api_key"]

# import os
# os.environ['OPENAI_API_KEY'] = gpt4_api_key

# ollama serve
# ollama list
# litellm --model ollama/llama2
config_list = [
    {
        "base_url": "http://127.0.0.1:8000",
        "api_key": "NULL",
        "model": "ollama/llama2",
    }
]

In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama

# llm = Ollama(model="llama2")

llm = Ollama(
    model="llama2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


Steps
1. Build up a vector store with Uniswap V3 whitepaper.

In [4]:
loaders = [ PyPDFLoader('../docs/uniswap_v3.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = "BAAI/bge-large-zh")

In [6]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=embedding_model
)
vectorstore.add_documents(docs)

['fb03a21a-b819-11ee-9677-00155dde3bcb',
 'fb03a27e-b819-11ee-9677-00155dde3bcb',
 'fb03a292-b819-11ee-9677-00155dde3bcb',
 'fb03a2a6-b819-11ee-9677-00155dde3bcb',
 'fb03a2ba-b819-11ee-9677-00155dde3bcb',
 'fb03a2ce-b819-11ee-9677-00155dde3bcb',
 'fb03a2d8-b819-11ee-9677-00155dde3bcb',
 'fb03a2ec-b819-11ee-9677-00155dde3bcb',
 'fb03a300-b819-11ee-9677-00155dde3bcb',
 'fb03a30a-b819-11ee-9677-00155dde3bcb',
 'fb03a31e-b819-11ee-9677-00155dde3bcb',
 'fb03a328-b819-11ee-9677-00155dde3bcb',
 'fb03a33c-b819-11ee-9677-00155dde3bcb',
 'fb03a350-b819-11ee-9677-00155dde3bcb',
 'fb03a364-b819-11ee-9677-00155dde3bcb',
 'fb03a36e-b819-11ee-9677-00155dde3bcb',
 'fb03a382-b819-11ee-9677-00155dde3bcb',
 'fb03a396-b819-11ee-9677-00155dde3bcb',
 'fb03a3a0-b819-11ee-9677-00155dde3bcb',
 'fb03a3b4-b819-11ee-9677-00155dde3bcb',
 'fb03a3be-b819-11ee-9677-00155dde3bcb',
 'fb03a3d2-b819-11ee-9677-00155dde3bcb',
 'fb03a3dc-b819-11ee-9677-00155dde3bcb',
 'fb03a3f0-b819-11ee-9677-00155dde3bcb',
 'fb03a3fa-b819-

2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [7]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [8]:
result = qa(({"question": "What is uniswap?"}))

/home/xiaodong/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Based on the provided context, Uniswap appears to be a decentralized exchange (DEX) platform built on top of the Ethereum blockchain. The platform allows users to swap two tokens for a third token in a trustless and non-custodial manner.

Uniswap v3 is the latest iteration of the platform, which introduces several new features such as concentrated liquidity, multiple pools per pair, improved Oracle upgrades, and more flexible governance. The platform also includes a factory contract that creates pools for each pair of tokens, with different swap fees for each pool.

The context provided does not reveal any information about Uniswap's underlying technology or architecture, but it does provide some insight into the platform's design and functionality.

In [9]:
result['answer']

"Based on the provided context, Uniswap appears to be a decentralized exchange (DEX) platform built on top of the Ethereum blockchain. The platform allows users to swap two tokens for a third token in a trustless and non-custodial manner.\n\nUniswap v3 is the latest iteration of the platform, which introduces several new features such as concentrated liquidity, multiple pools per pair, improved Oracle upgrades, and more flexible governance. The platform also includes a factory contract that creates pools for each pair of tokens, with different swap fees for each pool.\n\nThe context provided does not reveal any information about Uniswap's underlying technology or architecture, but it does provide some insight into the platform's design and functionality."


3. Define a function answer_uniswap_question

It takes a parameter question, calls the QA chain, and answer it by returning the answer from the chain response.


In [10]:
def answer_uniswap_question(question):
  response = qa({"question": question})
  return response["answer"]


4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [11]:
llm_config={
    "timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [14]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    # llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    # function_map={"answer_uniswap_question": answer_uniswap_question}
)

In [15]:
# next we register our functions, this is pretty straight forward
# we simply provide a dictionary mapping the names we expect the llm to call to references to the actual functions
# it's good practice to have the names the llm tries to call be the same as the actual pre-defined functions
# but we can call them whatever we want, 
user_proxy.register_function(
    function_map={
        "answer_uniswap_question": answer_uniswap_question
    }
)


It's time to start a task for the agents.

Now, let's user the user agent to ask the agents to write an introduction blog for Uniswap protocol v3.


In [16]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested tool Call (call_9e3ce006-95b9-45ca-924e-00558c1491a8):  *****
Arguments: 
{ "name": "answer_uniswap_question", "description": "Answer any Uniswap related questions", "parameters": { "type": "object", "properties": { "question": {"type": "string", "description": "The question to ask in relation to Uniswap protocol" } }, "required": ["question"] } }






































































































































































































































InternalServerError: Error code: 500 - {'error': {'message': 'can only join an iterable', 'type': None, 'param': None, 'code': 500}}